In [1]:
import pandas as pd
from src.utils.load_weights import load_weights
from src.utils.apply_ipfn import apply_ipfn
import numpy as np

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
poll_path = 'data/raw/Qriously_General_Election_2019.xlsx'
poll_df = pd.read_excel(poll_path, sheet_name='data')

In [3]:
weighting_variables = ['age', 'sex', 'NUTS1_region', 'constituency_type', 'education']

In [4]:
min_likelihood = 9

In [5]:
weighting_data = load_weights()

## Carry out the weighting using `ipfn`

It's expected that likelihood of voting (and possibly being registered to vote in the first place) correlates with some of the weighting variables. If we filtered first and then weighted, we would be undoing that relationship, so we need to **weight first, and then filter**.

In [6]:
weighting_data_subset = {k: v for k, v in weighting_data.items() if k in weighting_variables}

In [7]:
for var, targets  in weighting_data_subset.items():
    target_categories = targets.index.tolist()
    poll_categories = poll_df[var].unique()
    assert set(target_categories) == set(poll_categories), f'Mismatch between target categories and poll categories: {var}'

In [8]:
# remove the Weight column from the poll dataframe, as this is going to be added in the following cell
poll_df.drop('Weight', axis=1, inplace=True)

In [9]:
poll_df, marginal_check_dfs = apply_ipfn(poll_df, weighting_data_subset, check_marginals=True)

You can check that the weighted marginals the `ipfn` delivered match the ones we requested e.g.:

In [10]:
# marginal_check_dfs['sex']
# marginal_check_dfs['age']
# marginal_check_dfs['NUTS1_region']
# marginal_check_dfs['constituency_type']
# marginal_check_dfs['education']

In [11]:
for check_df in marginal_check_dfs.values():
    assert check_df['AbsDiff'].max() < 0.00001

Create a copy of the poll dataframe immediately after weighting, but before any filtering. This will help us with diagnostics later.

In [12]:
at_weighting_df = poll_df.copy()

## Filtering

Filter down to only registered voters, remove undecided/"I won't vote" people, and keep only reasonably likely voters

In [13]:
n_to_drop   = (poll_df['registered'] != 'Yes').sum()
pct_to_drop = (poll_df['registered'] != 'Yes').mean() * 100

print("Dropping %d rows (%.1f%% of data) for people not registered to vote" % (n_to_drop, pct_to_drop))

poll_df = poll_df.loc[poll_df['registered'] == 'Yes']

Dropping 295 rows (13.3% of data) for people not registered to vote


In [14]:
n_to_drop_undecided   = (poll_df['vote'] == 'Undecided').sum()
n_to_drop_not_voting  = (poll_df['vote'] == 'NotVoting').sum()

pct_to_drop_undecided  = (poll_df['vote'] == 'Undecided').mean() * 100
pct_to_drop_not_voting = (poll_df['vote'] == 'NotVoting').mean() * 100

sum_weight_drop_not_voting = poll_df[poll_df['vote'] == 'NotVoting']['Weight'].sum()

print("Dropping %d rows (%.1f%% of remaining data) for people who are undecided" % (n_to_drop_undecided, pct_to_drop_undecided))
print("Dropping another %d rows (another %.1f%% of remaining data) for people who chose 'I wouldn't vote'" % (n_to_drop_not_voting, pct_to_drop_not_voting))

poll_df = poll_df.loc[~ poll_df['vote'].isin(['Undecided', 'NotVoting'])]

Dropping 277 rows (14.4% of remaining data) for people who are undecided
Dropping another 193 rows (another 10.0% of remaining data) for people who chose 'I wouldn't vote'


In [15]:
weighted_pct_kept_numer = poll_df[poll_df['likelihood'] >= min_likelihood]['Weight'].sum()
weighted_pct_kept_denom = poll_df['Weight'].sum() + sum_weight_drop_not_voting

weighted_pct_kept = (weighted_pct_kept_numer / weighted_pct_kept_denom) * 100
print("Keeping %.1f%% of the most likely voters." % weighted_pct_kept)

Keeping 71.2% of the most likely voters.


In [16]:
poll_df = poll_df.loc[poll_df['likelihood'] >= min_likelihood]

## Diagnostics about weighting

We print effective sample sizes and weighting efficiences for (1). the full dataset, and (2). the data filtered to those we will use in the vote share prediction (i.e., registered likely decided voters). Using (1). is appropriate for thinking about any estimates computed on the whole dataset (such as what proportion of people are registered to vote, or likely to vote) whereas (2). is appropriate for the vote share estimates.

In [17]:
def effective_sample_size(series):
    top = series.sum() **2
    bottom = (series**2).sum()
    return top/bottom

**For the full dataset:**

In [18]:
at_weighting_df['Weight'].describe().to_frame()

,Weight
count,2222.000000
mean,1.000000
std,0.358217
min,0.407138
25%,0.738886
50%,0.943202
75%,1.195366
max,2.631251


In [19]:
ESS_full = effective_sample_size(at_weighting_df['Weight'])

print("Raw sample size: %d" % at_weighting_df.shape[0])
print("Effective sample size: %.1f" % ESS_full)

Raw sample size: 2222
Effective sample size: 1969.4


In [20]:
print("Weighting efficiency: %.1f%%" % ((ESS_full / at_weighting_df.shape[0]) * 100))

Weighting efficiency: 88.6%


**For the filtered dataset:**

In [21]:
poll_df['Weight'].describe().to_frame()

,Weight
count,1137.000000
mean,1.031582
std,0.356005
min,0.407138
25%,0.767085
50%,0.973697
75%,1.220266
max,2.592917


In [22]:
ESS = effective_sample_size(poll_df['Weight'])

print("Raw sample size: %d" % poll_df.shape[0])
print("Effective sample size: %.1f" % ESS)

Raw sample size: 1137
Effective sample size: 1016.1


In [23]:
print("Weighting efficiency: %.1f%%" % ((ESS / poll_df.shape[0]) * 100))

Weighting efficiency: 89.4%


## Estimate the national vote share!

In [24]:
weighted_vote_counts = poll_df.groupby('vote').agg(vote_count = ('Weight', 'sum'))['vote_count']
final_vote_share = ((weighted_vote_counts / weighted_vote_counts.sum()) * 100).to_frame('vote_pct') \
    .sort_values('vote_pct', ascending=False)

In [25]:
final_vote_share = final_vote_share.round(1)
display(final_vote_share)

,vote_pct
vote,
CON,43.2
LAB,30.4
LD,11.6
GRN,3.8
Other,3.4
BREX,3.2
SNP,2.3
DUP,0.8
SF,0.5


In [26]:
layout = {
    'yaxis': {'autorange': 'reversed'},
    'height': 700,
    'width': 1000,
    'colorway':['black']
}

figure = {
    'data': [
        {
            'type': 'bar',
            'y': final_vote_share.index,
            'x': final_vote_share['vote_pct'],
            'orientation': 'h',
        }],
    
    'layout': layout
}

iplot(figure)

# Analysis

Uncomment and run the cell below to take a look at the variables available for analysis

In [27]:
# print("Available variables:")
# for c in poll_df.columns:
#     print('\t',c)

In [28]:
# For columns generated by questions in the format 'What do you see as the most important issues facing Britain today?'...
# ... replace blank entries with text specifying the issue wasn't listed, to make crosstabs clearer
for c in poll_df.columns:
    if 'important_' in c:
        poll_df[c] = poll_df[c].fillna(value='Not listed')

In [29]:
def weighted_crosstab(var1, var2, weight_col='Weight'):
    '''Returns a crosstab, by default weighted by the 'Weight' column'''
    out = pd.crosstab(poll_df[var2], poll_df[var1], values=poll_df[weight_col], aggfunc = 'sum', normalize='index') * 100
    return out.round(0).astype('int8')

## Some examples

In [30]:
poll_df['simple_vote'] = poll_df['vote'].apply(lambda vote: vote if vote in ['LAB', 'CON', 'LD', 'BREX', 'SNP', 'GRN', 'DUP'] else 'All Others')

In [31]:
weighted_crosstab('simple_vote', 'age')

simple_vote,All Others,BREX,CON,DUP,GRN,LAB,LD,SNP
age,,,,,,,,
18-24,1,7,15,1,6,58,9,2
25-34,5,0,26,1,5,48,13,2
35-44,4,4,37,0,5,37,9,4
45-54,6,3,43,1,4,31,10,3
55-64,6,4,50,1,3,20,15,2
65+,5,4,61,1,2,14,12,2


In [32]:
weighted_crosstab('simple_vote', 'sex')

simple_vote,All Others,BREX,CON,DUP,GRN,LAB,LD,SNP
sex,,,,,,,,
Female,4,3,38,1,5,34,12,3
Male,5,3,48,1,3,27,12,2


In [33]:
weighted_crosstab('simple_vote', 'constituency_type')

simple_vote,All Others,BREX,CON,DUP,GRN,LAB,LD,SNP
constituency_type,,,,,,,,
Borough,3,4,39,1,4,38,10,1
County,6,3,46,1,4,25,12,3


In [34]:
weighted_crosstab('simple_vote', 'education')

simple_vote,All Others,BREX,CON,DUP,GRN,LAB,LD,SNP
education,,,,,,,,
no_quals,5,6,58,2,3,18,5,2
some_quals,4,3,51,1,2,27,10,2
university,5,2,30,0,5,39,16,3


## Your turn! 
Try replacing the first and second variables with things that are interesting to you.

In [ ]:
weighted_crosstab('simple_vote', 'important_immigration')

In [ ]:
weighted_crosstab('interest', 'employment_status')

In [ ]:
weighted_crosstab('important_immigration', 'employment_status')

In [ ]:
weighted_crosstab('important_poverty', 'best_leader')

In [ ]:
# weighted_crosstab('your_first_var_here','your_second_var_here')

All code is released under the MIT License.

Copyright 2020 Brandwatch

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.